In [14]:
import vastAIManager
import subprocess
import time
import importlib
importlib.reload(vastAIManager)

<module 'vastAIManager' from '/Users/kushgarg/Desktop/Programs/NeuralPerk/Neuralperk_ProxyUser_MainServer/service_VastAIService/vastAIManager.py'>

In [15]:
vast = vastAIManager.VastAIManager()

# Example filters: 1 GPU, at least 8GB GPU RAM, rentable
filters = ["num_gpus=1", "gpu_ram>=0", "rentable=true", "verified=true"]

cheapest = vast.get_cheapest_offer(filters)
print("Cheapest Offer:", cheapest)


Cheapest Offer: {'offer_id': '25084945', 'gpu': 'GTX_TITAN_X', 'price': '0.0414', 'dlperf': 'verified', 'reliability': 'South_Korea,_KR'}


In [17]:
def run_cheapest_instance_test(wait_time=1, filters=None):
    """
    Workflow: 
    - Get cheapest offer
    - Create instance
    - Start instance
    - Wait `wait_time` seconds
    - Stop + Destroy instance
    - Return cost summary
    """
    vast = vastAIManager.VastAIManager()

    # 1. Get cheapest offer
    cheapest = vast.get_cheapest_offer(filters)
    if not cheapest:
        vastAIManager.logging.error("No offers found. Aborting test.")
        return None

    offer_id = cheapest["offer_id"]

    # 2. Create instance
    contract_id = vast.create_instance(offer_id)
    if not contract_id:
        vastAIManager.logging.error("Failed to create instance.")
        return None

    # 3. Start instance (use contract_id)
    vast.start_instance(contract_id)

    # 4. Wait
    time.sleep(wait_time)

    # 5. Stop instance (use contract_id)
    vast.stop_instance(contract_id)

    # 6. Get summary BEFORE destroying instance (use contract_id)
    summary = vast.get_instance_summary(contract_id)
    vastAIManager.logging.info(f"Run summary: {summary}")

    # 7. Destroy instance (this removes it from tracking, use contract_id)
    vast.destroy_instance(contract_id)

    return summary


if __name__ == "__main__":
    filters = ["cuda_vers >= 12.0", "num_gpus=1", "gpu_ram>=8"]  # example filters
    summary = run_cheapest_instance_test(wait_time=1, filters=filters)
    print(summary)

{'ContractID': 25655852, 'OfferID': '22758225', 'Status': 'stopped', 'ComputeCost': 0.0, 'StorageCost': 0.0, 'TotalCost': 0.0, 'CreatedAt': 'Sat Sep  6 04:35:30 2025', 'StartedAt': 'Sat Sep  6 04:35:31 2025', 'StoppedAt': 'Sat Sep  6 04:35:34 2025'}


In [8]:
def run_docker_instance_test(dockerfile_content=None, docker_image=None, docker_command=None, wait_time=60, filters=None):
    """
    Test function to run Docker on a VastAI instance
    """
    if not filters:
        filters = ["num_gpus=1", "gpu_ram>=4", "rentable=true", "verified=true"]
    
    # 1. Find cheapest offer
    offers = vast.list_offers(filters)
    if not offers:
        return {"error": "No offers found"}
    
    cheapest = vast.get_cheapest_offer(filters)
    print(f"Selected offer: {cheapest}")
    
    # 2. Create instance
    contract_id = vast.create_instance(cheapest["offer_id"], disk=32)
    if not contract_id:
        return {"error": "Failed to create instance"}
    
    print(f"Instance created: {contract_id}")
    
    # 3. Start instance and wait for it to be ready (use contract_id)
    vast.start_instance(contract_id)
    print("Instance started, waiting for it to be ready...")
    time.sleep(30)  # Wait for instance to fully boot
    
    docker_results = {}
    summary = {}
    
    try:
        # 4. Check if Docker is available (most VastAI images have it pre-installed)
        print("Checking Docker environment...")
        
        # Check if docker command exists (use contract_id)
        docker_check = vast.execute_command_on_instance(contract_id, "which docker")
        if docker_check and "docker" in docker_check:
            print("Docker is already installed!")
        else:
            print("Docker not found, attempting to install...")
            setup_commands = [
                "apt-get update",
                "apt-get install -y docker.io",
                "systemctl start docker",
                "systemctl enable docker"
            ]
            
            for cmd in setup_commands:
                result = vast.execute_command_on_instance(contract_id, cmd)
                print(f"Setup command '{cmd}': {'Success' if result and 'error' not in result.lower() else 'Failed'}")

        # 5. Run Docker operations (use contract_id)
        print("Running Docker operations...")
        docker_results = vast.run_docker_on_instance(
            contract_id=contract_id,
            dockerfile_content=dockerfile_content,
            docker_image=docker_image,
            docker_command=docker_command
        )
        
        print("Docker Results:")
        for key, value in docker_results.items():
            print(f"  {key}: {value[:200] if isinstance(value, str) and len(value) > 200 else value}")

        # 6. Wait for specified time
        print(f"Waiting {wait_time} seconds...")
        time.sleep(wait_time)

    except Exception as e:
        print(f"Error during Docker operations: {e}")
        docker_results = {"error": str(e)}

    finally:
        # 7. Stop instance (use contract_id)
        vast.stop_instance(contract_id)
        print("Instance stopped")

        # 8. Get summary BEFORE destroying instance (use contract_id)
        summary = vast.get_instance_summary(contract_id)
        print(f"Cost summary: {summary}")

        # 9. Destroy instance (use contract_id)
        vast.destroy_instance(contract_id)
        print("Instance destroyed")

    return {
        "offer": cheapest,
        "docker_results": docker_results,
        "cost_summary": summary
    }

In [5]:
# Example 1: Simple Python script in Docker
simple_dockerfile = """
FROM python:3.9-slim
WORKDIR /app
RUN echo 'print("Hello from Docker!")' > hello.py
RUN echo 'print("GPU Info:")' >> hello.py
RUN echo 'import subprocess; subprocess.run(["nvidia-smi"])' >> hello.py
CMD ["python", "hello.py"]
"""

# Example 2: Blender rendering Dockerfile
blender_dockerfile = """
FROM nvidia/cuda:11.8-devel-ubuntu20.04
RUN apt-get update && apt-get install -y \\
    blender \\
    python3 \\
    python3-pip \\
    wget
WORKDIR /app
RUN echo 'import bpy; bpy.ops.render.render(); print("Render complete!")' > render_script.py
CMD ["blender", "--background", "--python", "render_script.py"]
"""

# Example 3: Custom NeuralPerk processing
neuralperk_dockerfile = """
FROM nvidia/cuda:11.8-devel-ubuntu20.04
RUN apt-get update && apt-get install -y \\
    python3 \\
    python3-pip \\
    git
WORKDIR /app
RUN pip3 install torch torchvision torchaudio
RUN echo 'import torch; print(f"PyTorch version: {torch.__version__}"); print(f"CUDA available: {torch.cuda.is_available()}"); print(f"GPU count: {torch.cuda.device_count()}")' > gpu_test.py
CMD ["python3", "gpu_test.py"]
"""

print("Docker examples ready!")

Docker examples ready!


In [ ]:
# Test 1: Run simple Python Docker container
print("=== Test 1: Simple Python Docker ===")
filters = ["num_gpus=1", "gpu_ram>=4", "rentable=true", "verified=true"]

# Uncomment to run:
result1 = run_docker_instance_test(
    dockerfile_content=simple_dockerfile,
    wait_time=60,
    filters=filters
)
print("Result:", result1)

=== Test 1: Basic Commands Debug ===
Selected offer: {'offer_id': '25244165', 'gpu': 'RTX_3060', 'price': '0.0414', 'dlperf': 'verified', 'reliability': 'South_Korea,_KR'}
Selected offer: {'offer_id': '25244165', 'gpu': 'RTX_3060', 'price': '0.0414', 'dlperf': 'verified', 'reliability': 'South_Korea,_KR'}
Instance created: 25655838
Instance created: 25655838
Waiting for instance to be ready...
Waiting for instance to be ready...
Testing extremely basic commands:
\nTrying: pwd
Testing extremely basic commands:
\nTrying: pwd
Result: failed with error 400: Invalid command given.

\nTrying: whoami
Result: failed with error 400: Invalid command given.

\nTrying: whoami
Result: failed with error 400: Invalid command given.

\nTrying: ls
Result: failed with error 400: Invalid command given.

\nTrying: ls
Result: failed with error 400: Execute command only avail on stopped instances. Use ssh to run commands on running instances.

\nTrying: echo hello
Result: failed with error 400: Execute comm

In [ ]:
# Test 1: Basic VastAI functionality
print("=== Testing VastAI Basic Functionality ===")

try:
    # Test getting offers
    filters = ["num_gpus=1", "gpu_ram>=4", "rentable=true", "verified=true"]
    offers = vast.list_offers(filters)
    print(f"Found {len(offers) if offers else 0} offers")
    
    if offers:
        cheapest = vast.get_cheapest_offer(filters)
        print(f"Cheapest offer: ${cheapest.get('dph_total', 'N/A')}/hour")
        print(f"Offer ID: {cheapest.get('offer_id', 'N/A')}")
        print(f"GPU: {cheapest.get('gpu_name', 'N/A')}")
        print(f"GPU RAM: {cheapest.get('gpu_ram', 'N/A')} GB")
    else:
        print("No offers found with current filters")
        
except Exception as e:
    print(f"Error testing basic functionality: {e}")

print("\n" + "="*50)

In [ ]:
# Test 2: Docker Integration Test
print("=== Testing Docker Integration ===")

try:
    # Run simple Docker test with minimal wait time
    result = run_docker_instance_test(
        dockerfile_content=simple_dockerfile,
        wait_time=10,  # Short wait time for testing
        filters=["num_gpus=1", "gpu_ram>=4", "rentable=true", "verified=true"]
    )
    
    print("Docker Test Results:")
    print(f"  Offer used: {result.get('offer', {}).get('offer_id', 'N/A')}")
    print(f"  Cost per hour: ${result.get('offer', {}).get('dph_total', 'N/A')}")
    
    if 'docker_results' in result:
        docker_res = result['docker_results']
        print(f"  Docker operations: {list(docker_res.keys())}")
        
        if 'run_container' in docker_res:
            container_output = docker_res['run_container']
            print(f"  Container output preview: {container_output[:200] if container_output else 'No output'}")
    
    if 'cost_summary' in result:
        cost = result['cost_summary']
        print(f"  Total cost: ${cost.get('total_cost', 'N/A')}")
        print(f"  Runtime: {cost.get('runtime_seconds', 'N/A')} seconds")
        
except Exception as e:
    print(f"Error in Docker integration test: {e}")

print("\n" + "="*50)

In [ ]:
# Run the basic test first to verify VastAI connection
print("Starting VastAI Integration Tests...")
print("Note: This will create, run, and destroy actual VastAI instances which will incur costs!")
print("Make sure you have VastAI CLI properly configured with your API key.")
print("")

# Uncomment the line below to run basic functionality test
# (This just lists offers and doesn't create instances, so it's safe)
# exec(open('testing.ipynb').read())  # Run the basic test cell above

In [13]:
# Test 2: Run GPU-enabled PyTorch container
print("=== Test 2: GPU PyTorch Docker ===")
gpu_filters = ["num_gpus=1", "gpu_ram>=8", "cuda_vers>=11.0", "rentable=true", "verified=true"]

# Uncomment to run:
# result2 = run_docker_instance_test(
#     dockerfile_content=neuralperk_dockerfile,
#     wait_time=120,
#     filters=gpu_filters
# )
# print("GPU Test Result:", result2)

=== Test 2: GPU PyTorch Docker ===


In [14]:
# Test 3: Run existing Docker image with custom command
print("=== Test 3: Existing Docker Image ===")

# Uncomment to run with existing image:
# result3 = run_docker_instance_test(
#     docker_image="nvidia/cuda:11.8-runtime-ubuntu20.04",
#     docker_command="nvidia-smi",
#     wait_time=30,
#     filters=gpu_filters
# )
# print("Existing Image Result:", result3)

=== Test 3: Existing Docker Image ===


In [18]:
# Quick test: Create instance and test basic command execution
print("=== Quick Command Test ===")
filters = ["num_gpus=1", "gpu_ram>=4", "rentable=true", "verified=true"]

# Find cheapest offer
offers = vast.list_offers(filters)
if offers:
    cheapest = vast.get_cheapest_offer(filters)
    print(f"Selected offer: {cheapest}")
    
    # Create and start instance
    offer_id = vast.create_instance(cheapest["offer_id"], disk=32)
    if offer_id:
        print(f"Instance created: {offer_id}")
        vast.start_instance(offer_id)
        print("Instance started, waiting 30 seconds...")
        time.sleep(30)
        
        # Test simple commands
        print("Testing basic commands:")
        
        commands_to_test = [
            "whoami",
            "pwd",
            "ls -la",
            "which docker",
            "docker --version"
        ]
        
        for cmd in commands_to_test:
            result = vast.execute_command_on_instance(offer_id, cmd)
            print(f"  '{cmd}': {result[:100] if result else 'No output'}")
        
        # Clean up
        vast.stop_instance(offer_id)
        vast.destroy_instance(offer_id)
        print("Instance cleaned up")
    else:
        print("Failed to create instance")
else:
    print("No offers found")

=== Quick Command Test ===
Selected offer: {'offer_id': '25244183', 'gpu': 'RTX_3060', 'price': '0.0481', 'dlperf': 'verified', 'reliability': 'South_Korea,_KR'}
Selected offer: {'offer_id': '25244183', 'gpu': 'RTX_3060', 'price': '0.0481', 'dlperf': 'verified', 'reliability': 'South_Korea,_KR'}
Instance created: 25655337
Instance started, waiting 30 seconds...
Instance created: 25655337
Instance started, waiting 30 seconds...
Testing basic commands:
  'whoami': No output
  'pwd': No output
  'ls -la': No output
  'which docker': No output
  'docker --version': No output
Instance cleaned up
Testing basic commands:
  'whoami': No output
  'pwd': No output
  'ls -la': No output
  'which docker': No output
  'docker --version': No output
Instance cleaned up
